In [1]:
!pip install kaggle

In [2]:
# !cp ../kaggle.json /root/.kaggle/kaggle.json
!

In [3]:
# Copying over from JH notebook
import os
from pathlib import Path
import pandas as pd
import numpy as np

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [4]:
import pandas as pd
import numpy as np
import copy
orig_train_df = pd.read_csv(path/'train.csv')
train_df = copy.deepcopy(orig_train_df)

In [5]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
# TODO: Check which ones JH included
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Parch', 'Fare']

In [7]:
num_df = train_df[num_vars].fillna(train_df[num_vars].mode().iloc[0])
num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)

# Going through an easier approach rather than typical norm
num_df = num_df / num_df.max()
num_df

,Age,SibSp,Parch,Fare
0,0.2750,0.125,0.000000,0.317521
1,0.4750,0.125,0.000000,0.683873
2,0.3250,0.000,0.000000,0.331789
3,0.4375,0.125,0.000000,0.636672
4,0.4375,0.000,0.000000,0.334298
...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118
887,0.2375,0.000,0.000000,0.545154
888,0.3000,0.125,0.333333,0.505672
889,0.3250,0.000,0.000000,0.545154


In [8]:
train_df[cat_vars].describe(include='all')
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
train_df.Pclass.unique()

array([3, 1, 2])

In [10]:
# Curious how in-depth you should conside N/As
cat_df = pd.get_dummies(train_df[cat_vars], columns=cat_vars)
cat_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,1,0,0,1
1,1,0,0,1,0,1,0,0
2,0,0,1,1,0,0,0,1
3,1,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,1,0,0,1,0,0,1
887,1,0,0,1,0,0,0,1
888,0,0,1,1,0,0,0,1
889,1,0,0,0,1,1,0,0


In [11]:
# Might need to normalize after creating the dummy vars?
proc_train_df = num_df.join(cat_df)

In [12]:
def process_df(path):
    df = pd.read_csv(path)
    df = df.fillna(df.mode().iloc[0])
    
    
    # TODO: Check which ones JH included
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    num_vars = ['Age', 'SibSp', 'Parch', 'Fare']
    
    
    num_df = df[num_vars]
    num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)
    # Going for an easier approach rather than typical norm
    num_df = num_df / num_df.max()

    cat_df = pd.get_dummies(df[cat_vars], columns=cat_vars)
    cat_df['const'] = 1
    
    return num_df.join(cat_df)

In [13]:
train_df = process_df(path/'train.csv')
train_df

/tmp/ipykernel_119/2267849073.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


In [14]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [15]:
test_df = process_df(path/'test.csv')
test_df

/tmp/ipykernel_119/2267849073.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.453947,0.000,0.000000,0.329840,0,0,1,0,1,0,1,0,1
1,0.618421,0.125,0.000000,0.311896,0,0,1,1,0,0,0,1,1
2,0.815789,0.000,0.000000,0.363976,0,1,0,0,1,0,1,0,1
3,0.355263,0.000,0.000000,0.346051,0,0,1,0,1,0,0,1,1
4,0.289474,0.125,0.111111,0.402083,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
414,0.513158,0.000,0.000000,0.751796,1,0,0,1,0,1,0,0,1
415,0.506579,0.000,0.000000,0.317521,0,0,1,0,1,0,0,1,1
416,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1


In [16]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [17]:
# Great success! We can move into building out model!

In [18]:
import torch

In [19]:
weights = torch.rand(train_df.shape[1]) - 0.5
weights

tensor([ 0.2476,  0.4658, -0.4095,  0.1200, -0.1773,  0.1669,  0.1509, -0.2728,
        -0.1946,  0.0155,  0.3278,  0.0437,  0.2237])

In [20]:
tensor_df = torch.tensor(train_df.values)
tensor_df

tensor([[0.2750, 0.1250, 0.0000,  ..., 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.0000],
        ...,
        [0.3000, 0.1250, 0.3333,  ..., 0.0000, 1.0000, 1.0000],
        [0.3250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.4000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000]],
       dtype=torch.float64)

In [21]:
result = (tensor_df * weights)
result

tensor([[ 0.0681,  0.0582, -0.0000,  ...,  0.0000,  0.0437,  0.2237],
        [ 0.1176,  0.0582, -0.0000,  ...,  0.0000,  0.0000,  0.2237],
        [ 0.0805,  0.0000, -0.0000,  ...,  0.0000,  0.0437,  0.2237],
        ...,
        [ 0.0743,  0.0582, -0.1365,  ...,  0.0000,  0.0437,  0.2237],
        [ 0.0805,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.2237],
        [ 0.0990,  0.0000, -0.0000,  ...,  0.3278,  0.0000,  0.2237]],
       dtype=torch.float64)

In [22]:
result.shape

torch.Size([891, 13])

In [23]:
train_df.shape

(891, 13)

In [24]:
def sigmoid(x):
    return 1 / (1 + 1 / torch.exp(-x))

In [25]:
torch_implementation_sigmoid = torch.sigmoid(result.sum(axis=1))
my_implementation_sigmoid = sigmoid(result.sum(axis=1))

In [26]:
# Awesome!
(torch_implementation_sigmoid == torch_implementation_sigmoid).sum()

tensor(891)

In [122]:
weights

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       requires_grad=True)

In [116]:
# TODO: Implement actual training

import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

def mse(preds, actual):
    return torch.abs(preds - actual).mean()

actual = torch.tensor(orig_train_df['Survived'])
weights = torch.rand(train_df.shape[1]) - 0.5
weights.requires_grad_()

def training_step(step, lr):
    global weights
    
    layer_results = tensor_df * weights
    preds = sigmoid(layer_results.sum(axis=1))
    loss = mse(preds, actual)

    
    if step % 100 == 0:
        print("loss:", loss)

    # print(weights.data)
        
    # if torch.isnan(loss):
    #     print(weights)
        
    loss.backward()
    
    with torch.no_grad():
        weights -= lr * weights.grad
        
def train(steps=10000, lr=3e-3):
    for i in range(steps):
        training_step(i, lr)
        
def pred(input_tensor):
    layer_results = input_tensor * weights
    return sigmoid(layer_results.sum(axis=1))

train()

loss: tensor(0.5500, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.3298, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2631, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2053, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2049, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2095, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2113, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2120, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2123, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2124, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2122, dtype=torch.float64, grad_fn=<MeanBackwar

In [112]:
weights.data

tensor([-0.0037,  0.2682, -0.4115, -0.3680, -0.1926,  0.1341, -0.0099,  0.3964,
        -0.0444,  0.1323, -0.1511, -0.0983, -0.4777])

In [77]:
weights - weights.grad

tensor([ 0.9238,  0.4809, -0.3831, -0.4528, -0.9741,  0.1857,  3.0801, -1.6432,
         4.3554, -0.1363,  0.0211,  2.3583,  1.8824], grad_fn=<SubBackward0>)

In [117]:
# Since weights are blowing up, going back through and checking my notebook against JH
# TODO: Plug in fastbook to write these up; learn in public bb
train_df

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


- Like the options to change display:

```python
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
```

- Doesn't look like he "normalized" numerical values
- cat_vars = [sex, pclass, embarked] which lines up with mine!
- num_vars line up as well, Age, SibSp, Parch, and Fare.
- Primary difference looks like he didn't
- Also his naming scheme looks a lot cleaner than mine; something to learn here
- I added a const column, might be better, but JH might've been just going the easy way

- Ahhhhh looks like problem might be `coeffs.grad.zero_()`

In [123]:
weights = torch.rand(train_df.shape[1]) - 0.5
tensor_df * weights

tensor([[-0.0911, -0.0258,  0.0000,  ...,  0.0000, -0.0806,  0.0529],
        [-0.1573, -0.0258,  0.0000,  ...,  0.0000, -0.0000,  0.0529],
        [-0.1076, -0.0000,  0.0000,  ...,  0.0000, -0.0806,  0.0529],
        ...,
        [-0.0993, -0.0258,  0.0062,  ...,  0.0000, -0.0806,  0.0529],
        [-0.1076, -0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0529],
        [-0.1325, -0.0000,  0.0000,  ...,  0.3742, -0.0000,  0.0529]],
       dtype=torch.float64)

In [124]:
weights.shape

torch.Size([13])